This notebook is part of the youtube video "Jet Engine Series" from the Flight Test Engineering Channel: follow this [link](https://youtube.com/@flighttestengineering)

Episode 1: Series Intro

Episdoe 2: Inlet Thermodynamics

Episode 3: Inlet Python Coding

Episode 4: *Compressor Thermodynamics* - use this notebook

# A Small Recap

Ideal Gas

Isentropic Process

* $pV^{\gamma}=k$; $p=k\rho^{\gamma}$ -> $dp=\gamma k \rho^{\gamma -1}d\rho$

Isentropic Compression

* $\frac{T_2}{T_1}=(\frac{p_2}{p_1})^{\frac{\gamma-1}{\gamma}}$  >>>or<<<   $\frac{p_2}{p_1}=(\frac{T_2}{T_1})^{\frac{\gamma}{\gamma-1}}$


Bernoulli / Compressible 

* $\frac{p_0}{p} = (1+ \frac{\gamma-1}{2} M^2)^{\frac{\gamma}{\gamma - 1}} = (1+ \frac{V^2}{2 c_p T_s})^{\frac{\gamma}{\gamma - 1}}$

Compression Efficiency

* $\eta_c = \frac{w_{c,s}}{w_c} = \frac{h_{2}^\prime-h_1}{h_2-h_1} = \frac{c_p(T_{2}^\prime-T_1)}{c_p(T_2-T_1)}$

![037](pictures/compression_efficiency.svg)


Some basic relationships

$M=\frac{V}{\sqrt{\gamma RT_{static}}}$

$\gamma R = c_p (\gamma - 1)$

# Inlet Diffuser

The conditions far away from the engine, at Station "a" (for ambient), are:

$T_{a_{static}}$, $P_{a_{static}}$, and $M_a$

If we now define *sub "0"* to mean "stagnation"...

so $T_{01}$ is the stagnation pressure at station 1;

   $T_{0a}$ is the stagnation pressure at station "a"

and $T_{a}$ is the static pressure at station "a"

#### Isentropic flow up to duct inlet

Let's assume there are no losses from Station "a" to Station 1.

$P_{01} = P_{0a} = P_{a} (1 + \frac{\gamma - 1}{2}M_a^{2})^\frac{\gamma}{\gamma -1} = P_{a} (1+ \frac{V_{a}^2}{2 c_p T_a})^{\frac{\gamma}{\gamma - 1}}$

$T_{01} = T_{0a} = T_{a} (1 + \frac{\gamma - 1}{2}M_a^{2}) = T_{a}+ (\frac{V_{a}^2}{2 c_p} )$

#### Adiabatic Duct

Inside the inlet duct, we have friction losses, but no heat transfer

$T_{02}=T_{01}=T_{0a} $ -> stagnation temperatures are the same

$=> T_{02}=T_{01}= T_{1} + \frac{V_1^2}{2c_{p1}}$

$T_{02} - T_{1} =  \frac{V_1^2}{2c_{p1}} $

$=> T_{1} + \frac{V_1^2}{2c_{p1}} = T_{2} + \frac{V_2^2}{2c_{p2}} $

$T_{2} = T_{1} + \frac{V_1^2}{2c_{p1}} - \frac{V_2^2}{2c_{p2}}$

#### if it was isentropic...

And if we had isentropic compression, we could write:

$\frac{p_{02}}{p_{1}} = (\frac{T_{02}^\prime}{T_{1}})^{\frac{\gamma}{\gamma -1}}$

which can be re-written as:

$\frac{p_{02}}{p_{1}} = (1 +  \frac{T_{02}^{'} - T_{1}}{T_{1}})^{\frac{\gamma}{\gamma -1}}$



#### considering the definition of inlet efficiency...



$\eta_i=\frac{T_{02}^{'}-T_{1}}{T_{02}-T_{1}}$

$T_{02}^{'}-T_{1}=\eta_i(T_{02}-T_{1})$ 

$\frac{p_{02}}{p_{1}} = (1 +  \frac{\eta_i(T_{02}-T_{1})}{T_{1}})^{\frac{\gamma}{\gamma -1}}$

But $T_{02} - T_{1} =  \frac{V_1^2}{2c_{p1}} $


$\frac{p_{02}}{p_{1}} = (1 + \frac{\eta_i V_1^2}{2c_{p1}T_{1}})^{\frac{\gamma}{\gamma -1}}$

#### mass flow

$\dot{m}=\rho V A$

In [ ]:
#01 - preamble, imports

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import cantera as ct
import ISA_module as ISA

from engine_helper_functions import *

plt.rcParams['figure.figsize'] = [12, 7]

In [ ]:
ct.__version__

In [ ]:
#02 - engine parameters definitions

eng_param = {} # engine physical parameters will be stored in this dictionary

eng_perf = {} # engine performance parameters in this dictionary

# engine parameters
# inlet
#   physical parameters
eng_param['A1'] = 0.30 # m2 inlet area at station 1
eng_param['A2'] = 0.32 # m2 inlet exit area at face of compressor
#   performance parameters
eng_perf['eta_i'] = 0.98 # this can be upgraded to vary with Mach and m_dot

#--------------------------------------------------------------------------

# compressor
#   physical parameters
eng_param['comp_n_stages'] = 10 # number of stages in compressor
#   performance parameters
eng_perf['CPR'] = 6.1  # overall compressor pressure rise
eng_perf['eta_c'] = 0.80 # isentropic stage efficiency for compressor


In [ ]:
#03 - inlet iterative function

def iterate_inlet(mdot:float, 
                  A:float, 
                  gas_in:ct.Solution, 
                  eta_i:float, 
                  M_in:float, 
                  gas_out:ct.Solution)->(float, bool):
    '''
    This function iterates the gas velocity at inlet exit until convergence
    
    inputs
    mdot : mass flow, in kg/s
    A     : area, in m2
    gas_in: Cantera solution object with gas at entrance of inlet
    T_in  : stagnation temperature of gas at entrance of inlet, in K
    p_in  : stagnation pressure of gas at entrance of inlet, in Pascals
    eta_i : inlet efficiency
    M_in  : Mach number for gas at entrance of inlet
    gas_out: Cantera solution object with gas at exit of inlet
    
    returns
    M_out : Mach number for gas at exit of inlet. Zero if no convergence reached
    convergence: True if converged, False if not
    indirect outputs
    gas_out: Cantera solution containing updated gas properties
    '''
    # loop control
    tol = 0.01 # tolerance to check for convergence
    max_iter = 100 # maximum number of iterations
    converged = False # keeps track of convergence
    n_iter = 0 #iteration counter
    
    # calculated input gas properties
    V_in = M_in * get_a(gas_in)
    gamma_in = get_gamma(gas_in)
    T_0in = get_T(gas_in.T, gamma_in, M_in)
    p_0in = get_p(gas_in.P, gamma_in, M_in)

    # initial guess
    T_0out = T_0in
    V_out_guess = mdot / (gas_in.density * A)
    gamma_out = gamma_in
    
    
    while not converged and n_iter <= max_iter:
        
        # calc properties using current guess
        
        T_out = gas_in.T + (V_in**2 / (2 * gas_in.cp) - V_out_guess**2 / (2 * gas_out.cp))
        p_0out = gas_in.P * (1 + eta_i * V_in**2 / (2 * gas_in.cp * gas_in.T))**(gamma_in / (gamma_in - 1))
        p_out = p_0out * (T_out / T_0out)**(gamma_out / (gamma_out - 1))
              
        # update gas to get new properties (especially density)
        gas_out.TP = T_out, p_out
        gamma_out = get_gamma(gas_out)
        
        # update velocity calculation with new gas properties
        V_out = V_out_guess
        V_out_guess = mdot / (gas_out.density * A)
        
        # check for convergnece
        if abs(V_out - V_out_guess) < tol:
            print(f'inlet finished, converged, niter={n_iter}')
            converged = True
            M_out = V_out / get_a(gas_out)
        elif n_iter < max_iter:
            n_iter += 1
        else:
            M_out = 0
            print(f'inlet finished, NOT converged, niter={n_iter}')
            
    return M_out, converged            

# COMPRESSOR

## compressor efficiency...



$\eta_c=\frac{T_{03}^{'}-T_{02}}{T_{03}-T_{02}}$

$T_{03}-T_{02}=\frac{1}{\eta_c}(T_{03}^{'}-T_{02})=\frac{T_{02}}{\eta_c}(\frac{T_{03}^{'}}{T_{02}}-1)=\frac{T_{02}}{\eta_c}[(\frac{p_{03}}{p_{02}})^{\frac{\gamma-1}{\gamma}}-1]$

$T_{03}=T_{02}+\frac{T_{02}}{\eta_c}[(\frac{p_{03}}{p_{02}})^{\frac{\gamma-1}{\gamma}}-1]$


Multi-stage compressors

$(CR_{stage})^{n}=CPR$

$CR_{stage}=CPR^{\frac{1}{n}}$

In [ ]:
#08 - multi-stage compressor - "constant" temp rise

def multi_stage_compressor(gas_in:ct.Solution, 
                          n_stages:float, 
                          CPR:float, 
                          eta_c:float, 
                          M_in:float,
                          gas_out:ct.Solution):
    '''
    This function calculates the output properties of a multi-stage axial compressor
    
    inputs
    gas_in    : Cantera solution object with gas at entrance of inlet
    n_stages  : number of stages in axial compressor
    CPR       : overall compression ratio
    eta_c     : stage isentropic efficiency
    M_in      : Mach number for gas at entrance of inlet
    gas_out   : Cantera solution object with gas at exit of inlet
    
    returns
    T_out, p_out : list with static temperatures, static pressures of each stage
    convergence  : True if converged, False if not
    compressor_work : specific work used to compress gas [in kJ/kg]
    
    indirect outputs
    gas_out: Cantera solution containing updated gas properties
    '''
    
    # calculated input gas properties
    gamma_in = get_gamma(gas_in)
    T_0in = get_T(gas_in.T, gamma_in, M_in) # stagnation
    p_0in = get_p(gas_in.P, gamma_in, M_in) # stagnation
    
    # pressure ratio per stage
    CR_stage = CPR**(1 / n_stages)
    
    
    # gradually shift pressure towards the initial stages
    stage_multiplier = np.ones(n_stages)
    shift = 0.001 # overall shift amount, per iteration
    shifter = np.ones(n_stages)
    step_shift = shift / n_stages # shift step per stage
    
    center = int(n_stages / 2) # middle of vector - it does not matter if we are off by one (even length vector)
    for i in range(center):
        shifter[i] = 1 + (center - i) * step_shift # shift initial stages UP
        shifter[n_stages - i - 1] = 1 - ((center - i) * step_shift) # shift later stages DOWN
    
    # pressure rise shift loop control
    converged = False
    n_iter = 0
    max_iter = 5000
    prev_delta_t = 1000 # start with high value to trigger condition
    
    # stage properties
    stages_p_out = np.zeros(n_stages) # holds the press data for each stage
    stages_T_out = np.zeros(n_stages) # holds the temp data for each stage
    stage_gas = (ct.Solution(reaction_mechanism, phase_name)) # internal object to keep track of gas properties
    stage_gas.X = comp_air
    
    # compressor output data
    compressor_work = 0 # collector for specific work used to compress gas, for all stages, in kJ/kg

    
    # pressure rise shift loop
    while not converged and n_iter <= max_iter:

        stage_gas.TP = gas_in.T, gas_in.P
        
        # thermo stages loop
        for st_counter in range(n_stages):

            T_i = stage_gas.T # keep initial temperature for work calculation
            
            gamma = get_gamma(stage_gas)
            p0 = get_p(stage_gas.P, gamma, M_in) * CR_stage * stage_multiplier[st_counter]
            T0 = T_0in / eta_c *((p0 / p_0in)**((gamma - 1) / gamma) - 1) + T_0in

            T = get_Ts(T0, gamma, M_in)
            p = get_ps(p0, T, T0, gamma)
            stage_gas.TP = T, p # do an update on TP, with previous gamma

            gamma = get_gamma(stage_gas) # update gamma
            T = get_Ts(T0, gamma, M_in)
            p = get_ps(p0, T, T0, gamma)
            stage_gas.TP = T, p # refine TP with updated gamma
            
            # store conditions for plotting later...
            stages_p_out[st_counter] = p
            stages_T_out[st_counter] = T
            
            # store stage work
            compressor_work += stage_gas.cp * (T - T_i) # in kJ/kg

            # update for next stage
            p_0in = p0
            T_0in = T0

        # logic to account for different number of stages
        if n_stages > 2: # typical multi-stage case
            max_delta_t = np.diff(stages_T_out).max()
        elif n_stages > 1: # special case : np.diff will drop one in vector length
            max_delta_t = max(stages_T_out[1] - stages_T_out[0], stages_T_out[0] - gas_in.T)
        else: # case for 1 stage
            max_delta_t = T - gas_in.T
        

        # loop objective is to get minimum temperature difference between all stages
        # by shifting pressure rise towards initial stages
        if max_delta_t < prev_delta_t and n_iter < max_iter:

            n_iter += 1
            # clear previous data and reset inputs
            T_0in = get_T(gas_in.T, gamma_in, M_in)
            p_0in = get_p(gas_in.P, gamma_in, M_in)
            compressor_work = 0 #zero out work absorbed by compressor
            
            # increase pressure shift towards initial stages
            stage_multiplier = np.multiply(stage_multiplier, shifter)
            prev_delta_t = max_delta_t
        
        elif n_iter >= max_iter:
            print(f'compressor finished, NOT converged, niter={n_iter}, max delta T={max_delta_t:0.1f}')
            n_iter += 1
        else:
            converged = True
            print(f'compressor finished, converged, niter={n_iter}, max delta T={max_delta_t:0.1f}')
        
        
    # update gas_out to pass properties back
    gas_out.TP = T, p


    return list(zip(stages_T_out, stages_p_out)), converged, compressor_work

In [ ]:
#04 - engine ambient/operating conditions

eng_op_con = {}
eng_op_con['throttle_pos'] = 1.0 # 1 is full throttle; .5 is idle
eng_op_con['mdot_guess'] = 20 # kg/s
eng_op_con['alt'] = 35000 # ft
eng_op_con['M_i'] = 0.8

M_i = eng_op_con['M_i'] # indicated Mach number - aircraft
V_i = ISA.M2Vt(eng_op_con['M_i'], eng_op_con['alt']) * ISA.kt2ms # true airspeed in kts
p_amb = ISA.p(eng_op_con['alt']) # static
T_amb = ISA.T(eng_op_con['alt']) # static

In [ ]:
#05 - initial stations setup

gas = {} # dictionary with Cantera Solution (gas) objects for each station
M = {} # Mach number for each station

st = ["a", 1, 2, 3, 4, 5, 8] # station numbers
station_names = {st[0]:'ambient',
                 st[1]:'inlet',
                 st[2]:'inlet @ comp. face',
                 st[3]:'after compressor',
                 st[4]:'after combustor',
                 st[5]:'after turbine',
                 st[6]:'nozzle exit'}

# see https://github.com/Cantera/cantera/blob/main/data/nDodecane_Reitz.yaml
reaction_mechanism = 'nDodecane_Reitz.yaml'
phase_name = 'nDodecane_IG' # IG = ideal gas, other option is RK = Redlich-Kwong

comp_air = 'O2:0.209, N2:0.787, CO2:0.004' # composition of air
comp_fuel = 'c12h26:1' # composition of fuel


# initialize all stations with air, at ambient conditions
for station in st:
    gas[station] = (ct.Solution(reaction_mechanism, phase_name))
    gas[station].X = comp_air
    gas[station].TP = T_amb, p_amb
        
    M[station] = M_i

In [ ]:
#06 - from Station "a" to Station 1

M_calc, conv = iterate_inlet(eng_op_con['mdot_guess'],
                             eng_param['A1'],
                             gas[st[0]],
                             1, # isentropic thus efficiency=1
                             M[st[0]],
                             gas[st[1]])

# we only assign the Mach number if we reached convergnece
if conv: M[st[1]] = M_calc

In [ ]:
# create data set for plotting

st_T = []
st_p = []
st_X = []

# get T, p, X from each station that we already calculated
for x in range(0, 2):
    st_T.append(gas[st[x]].T)
    st_p.append(gas[st[x]].P)
    st_X.append(gas[st[x]].X)

In [ ]:
print_stations(st[0:2], station_names, gas, M)

myplot = plot_T_s(st_T, st_p, st_X, reaction_mechanism, phase_name)

In [ ]:
#07 - from Station 1 to Station 2

M_calc, conv = iterate_inlet(eng_op_con['mdot_guess'] ,
                             eng_param['A2'],
                             gas[st[1]],
                             eng_perf['eta_i'],
                             M[st[1]],
                             gas[2])

# we only assign the Mach number if we reached convergnece
# and if we did, we now assume constant Mach throughout the machine
if conv: 
    n_st = len(st) #get number of stations
    current_st = st.index(2) #get index of current station
    for i in range(current_st, n_st):
        M[st[i]] = M_calc
else:
    print('ERROR: inlet did not converge')
    ###break

st_T.append(gas[st[2]].T)
st_p.append(gas[st[2]].P)
st_X.append(gas[st[2]].X)

In [ ]:
print_stations(st[0:3], station_names, gas, M)

myplot = plot_T_s(st_T, st_p, st_X, reaction_mechanism, phase_name)

In [ ]:
#09 - calculate compressor exit station

st_out, conv, compressor_work = multi_stage_compressor(gas[2], 
                                eng_param['comp_n_stages'], 
                                eng_perf['CPR'], 
                                eng_perf['eta_c'], 
                                M[st[2]], 
                                gas[st[3]])



# add T, p, X from compressor stages
for x in st_out:
    st_T.append(x[0])
    st_p.append(x[1])
    st_X.append(st_X[2]) # we just add a fixed value here because the composition in the compressor is not changing


In [ ]:
print_stations(st[0:4], station_names, gas, M)

myplot = plot_T_s(st_T, st_p, st_X, reaction_mechanism, phase_name)

In [ ]:
# pressure ratio per stage
n_stages = 10
CPR = 6.1
CR_stage = CPR**(1 / n_stages)


# gradually shift pressure towards the initial stages
stage_multiplier = np.ones(n_stages)
shift = 0.001 # overall shift amount, per iteration
shifter = np.ones(n_stages)
step_shift = shift / n_stages # shift step per stage

center = int(n_stages / 2) # middle of vector - it does not matter if we are off by one (even length vector)
for i in range(center):
    shifter[i] = 1 + (center - i) * step_shift # shift initial stages UP
    shifter[n_stages - i - 1] = 1 - ((center - i) * step_shift) # shift later stages DOWN


In [ ]:
print(shifter)

In [ ]:
f_shifter = np.ones((shifter.shape[0], 101))

In [ ]:
for i in range(1,101):
    f_shifter[:,i] = f_shifter[:,i-1]*shifter

In [ ]:
f_shifter[:,1]

In [ ]:
for i in range(0,101,10):
    plt.plot(f_shifter[:,i], label=i)
    
plt.legend();
plt.show;